In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from collections import OrderedDict
import sys
import os

sys.path.append('/home/christoph/Dokumente/christoph-MA/MA-Repo')
import Aggregator_Module
import Dataloader_patches_aggregator
from train import train_model
from eval import evaluate_model
from torchvision.models import video

/home/christoph/Dokumente/christoph-MA/MAenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
SwinTransformer = video.swin3d_b(video.Swin3D_B_Weights.KINETICS400_V1)
SwinTransformer.head = nn.Linear(SwinTransformer.head.in_features, 3)

SwinEncoder = video.swin3d_b(video.Swin3D_B_Weights.KINETICS400_V1)
SwinEncoder.head = nn.Identity()


state_dict = torch.load('/home/christoph/Dokumente/christoph-MA/Models/swin_transformer_3D_organ_classification_patches_tripath.pth')

SwinTransformer = nn.DataParallel(SwinTransformer)
SwinTransformer = SwinTransformer.to("cuda")

# Load the modified state dictionary into the model
SwinTransformer.load_state_dict(state_dict)

# Set the model to evaluation mode
SwinTransformer.eval()

/home/christoph/Dokumente/christoph-MA/MAenv/lib/python3.12/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


DataParallel(
  (module): SwinTransformer3d(
    (patch_embed): PatchEmbed3d(
      (proj): Conv3d(3, 128, kernel_size=(2, 4, 4), stride=(2, 4, 4))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (features): Sequential(
      (0): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): ShiftedWindowAttention3d(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (proj): Linear(in_features=128, out_features=128, bias=True)
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (0): Linear(in_features=128, out_features=512, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=512, out_features=1

In [4]:
decoder_enc = nn.Sequential(
        nn.Linear(1024, 128),
        nn.GELU(),
        nn.Dropout(0)
    )

    # Initialize the aggregator model
aggregator = Aggregator_Module.AttnMeanPoolMIL(gated=True, dropout=0, out_dim=3, encoder=decoder_enc, encoder_dim=128)
aggregator.start_attention(freeze_encoder=False)

state_dict = torch.load('/home/christoph/Dokumente/christoph-MA/Models/SwinTransformer_Aggregator_3D_organ_classification_patches_tripath_False.pth')

aggregator = nn.DataParallel(aggregator)
aggregator = aggregator.to("cuda")

aggregator.load_state_dict(state_dict=state_dict)

<All keys matched successfully>

In [51]:
import numpy as np
mask_path = '/storage/Datensätze/Leber_2,75um/Segmentation.raw'

volume_path = '/storage/Datensätze/Leber_2,75um/Datenbank_Leber_1_2-75um_2.75244_1987x1987x1235.raw'

shape = (1987, 1987, 1235)

volume = np.fromfile(volume_path, dtype=">u2").reshape(shape, order='F')

mask_shape = (shape[0] // 4, shape[1] // 4, shape[2] // 4)

mask = np.fromfile(mask_path, dtype=">u2").reshape(mask_shape, order='F')

In [52]:

def extract_patches(volume, mask, patch_size=(128, 128, 32), threshold=0.01):
    patch_height, patch_width, patch_depth = patch_size

    assert volume.shape[0] // 4 == mask.shape[0] and volume.shape[1] // 4 == mask.shape[1] and volume.shape[2] // 4 == mask.shape[2]

    patches = []
    for i in range(0, volume.shape[0], patch_height):
        for j in range(0, volume.shape[1], patch_width):
            for k in range(0, volume.shape[2], patch_depth):
                start_x = i
                end_x = min(start_x + patch_height, volume.shape[0])
                start_y = j
                end_y = min(start_y + patch_width, volume.shape[1])
                start_z = k
                end_z = min(start_z + patch_depth, volume.shape[2])

                if end_x - start_x < patch_height:
                    start_x = end_x - patch_height
                if end_y - start_y < patch_width:
                    start_y = end_y - patch_width
                if end_z - start_z < patch_depth:
                    start_z = end_z - patch_depth

                volume_patch = volume[start_y:end_y, start_x:end_x, start_z:end_z]
                mask_patch = mask[start_y // 4:end_y // 4, start_x // 4:end_x // 4, start_z // 4:end_z // 4]

                if np.mean(mask_patch) >= threshold:
                    patches.append(volume_patch)

    return patches

In [53]:
patches = extract_patches(volume, mask, patch_size=(128, 128, 32), threshold=0.01)
patches = np.array(patches, dtype=np.float32)
# transform np patch array to torch tensor
patches = torch.from_numpy(patches)  # Add a channel dimension
patches.unsqueeze_(1)  # Add a channel dimension
patches = patches.repeat(1, 3, 1, 1, 1)  # Repeat the tensor along the channel dimension
print(f"Shape of patches: {patches.shape}")

Shape of patches: torch.Size([6873, 3, 128, 128, 32])


In [54]:
batch_size = 16  # Adjust batch size based on available GPU memory
output = []

for i in range(0, len(patches), batch_size):
    batch = patches[i:i + batch_size].to("cuda")  # Move a batch of patches to GPU
    with torch.no_grad():  # Disable gradient computation to save memory
        batch_output = SwinTransformer(batch)
    output.extend(batch_output.cpu())  # Move output back to CPU to free GPU memory
    torch.cuda.empty_cache()  # Clear CUDA cache

In [57]:
# for every element in output, get the max value and the index of the max value
output = torch.stack(output)
output = output.cpu()
output = output.numpy()
class_pred = np.argmax(output, axis=1)

class_pred_probs = np.mean(output, axis=0)
print(class_pred_probs)
import torch.nn.functional as F
print(F.softmax(torch.tensor(class_pred_probs), dim=0))


TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not numpy.ndarray

In [56]:
#give the number of predictions for each class
num_classes = 3
class_counts = np.zeros(num_classes)
for i in range(num_classes):
    class_counts[i] = np.sum(class_pred == i)
print(f"Class counts: {class_counts}")

Class counts: [6873.    0.    0.]


In [57]:
#average the predicted provabilities for each class
average_probabilities = np.zeros(num_classes)
for i in range(num_classes):
    average_probabilities[i] = np.mean(predicted_probabilities[predicted_classes == i])
print(f"Average probabilities: {average_probabilities}")

Average probabilities: [11.33199024         nan         nan]


/home/christoph/Dokumente/christoph-MA/MAenv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/christoph/Dokumente/christoph-MA/MAenv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
batch_size = 16  # Adjust batch size based on available GPU memory
features = []

SwinEncoder = SwinEncoder.to("cuda:0")

for i in range(0, len(patches), batch_size):
    batch = patches[i:i + batch_size].to("cuda")  # Move a batch of patches to GPU
    with torch.no_grad():  # Disable gradient computation to save memory
        batch_output = SwinEncoder(batch)
    features.extend(batch_output.cpu())  # Move output back to CPU to free GPU memory
    torch.cuda.empty_cache()  # Clear CUDA cache

In [ ]:
#make features a tensor
features = torch.stack(features)

In [ ]:
features = features.unsqueeze(1)

In [ ]:
batch_size = 16  # Adjust batch size based on available GPU memory
output = []

for i in range(0, len(features), batch_size):
    batch = features[i:i + batch_size].to("cuda")  # Move a batch of patches to GPU
    with torch.no_grad():  # Disable gradient computation to save memory
        batch_output = aggregator(batch)
    output.extend(batch_output.cpu())  # Move output back to CPU to free GPU memory
    torch.cuda.empty_cache()  # Clear CUDA cach

In [ ]:
# for every element in output, get the max value and the index of the max value
output = torch.stack(output)
output = output.cpu()
output = output.numpy()
class_pred = np.argmax(output, axis=1)

class_pred_probs = np.mean(output, axis=0)
print(class_pred_probs)
import torch.nn.functional as F
print(F.softmax(torch.tensor(class_pred_probs), dim=0))


[ 10.2823    -11.06502    -0.7754286]
tensor([9.9998e-01, 5.3576e-10, 1.5765e-05])


In [50]:
print(class_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 